<a href="https://colab.research.google.com/github/esb-index/Barka-AV/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import xarray as xr
xr.open_dataset("/content/era5_extracted/dania4/data_stream-oper_stepType-instant.nc")


<xarray.Dataset> Size: 7MB
Dimensions:     (valid_time: 2191, latitude: 6, longitude: 22)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 18kB 2018-01-01T12:00:00 ... 2023...
  * latitude    (latitude) float64 48B 56.65 56.4 56.15 55.9 55.65 55.4
  * longitude   (longitude) float64 176B 7.3 7.55 7.8 8.05 ... 12.05 12.3 12.55
    number      int64 8B ...
    expver      (valid_time) <U4 35kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 1MB ...
    v10         (valid_time, latitude, longitude) float32 1MB ...
    t2m         (valid_time, latitude, longitude) float32 1MB ...
    msl         (valid_time, latitude, longitude) float32 1MB ...
    sst         (valid_time, latitude, longitude) float32 1MB ...
    sd          (valid_time, latitude, longitude) float32 1MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-10-13T07:15 GRIB to CDM+CF via cfgrib-0.9.1...

In [1]:
# ==========================
# ERA5 "light-RAM" feldolgozó pipeline
# ==========================
# minden .zip már ki van bontva /content/era5_extracted alá

!pip install xarray netCDF4 pandas numpy tqdm

import xarray as xr
import pandas as pd
import numpy as np
import glob, os
from tqdm import tqdm

EXTRACT_DIR = "/content/era5_extracted"
OUT_DIR = "/content/era5_processed_light"
os.makedirs(OUT_DIR, exist_ok=True)

def region_from_path(p):
    p_low = p.lower()
    if "dania" in p_low: return "dania"
    if "nemet" in p_low: return "nemet"
    if "uk" in p_low: return "uk"
    if "tajvan" in p_low: return "tajvan"
    if "usa" in p_low: return "usa"
    return "other"

all_files = sorted(glob.glob(os.path.join(EXTRACT_DIR, "**", "*.nc"), recursive=True))
region_files = {}
for f in all_files:
    r = region_from_path(f)
    region_files.setdefault(r, []).append(f)

print({r: len(v) for r,v in region_files.items()})

# ---- kis segédfüggvények ----
def safe_open_dataset(path):
    """megnyitja a NetCDF-et és átnevezi valid_time -> time"""
    ds = xr.open_dataset(path)
    if "valid_time" in ds.dims and "time" not in ds.dims:
        ds = ds.rename({"valid_time":"time"})
    ds = ds.chunk({"time":200})
    return ds

def winsorize(arr, low=1, high=99):
    lo = np.nanpercentile(arr, low)
    hi = np.nanpercentile(arr, high)
    return np.clip(arr, lo, hi)

# ---- fő feldolgozó ciklus ----
for region, files in region_files.items():
    if region == "other" or len(files)==0:
        continue
    print(f"\n=== Feldolgozás: {region} ===")

    region_dailies = []
    for f in tqdm(files, desc=f"{region} fájlok"):
        try:
            ds = safe_open_dataset(f)
        except Exception as e:
            print(f"Hiba {f}: {e}")
            continue

        # napi átlag, sum, max
        daily = xr.Dataset()
        for v in ds.data_vars:
            data = ds[v]
            if "time" not in data.dims:
                continue
            if v.lower().startswith(("tp","precip")):
                daily[v+"_sum"] = data.resample(time="1D").sum()
            elif v.lower().startswith(("u10","v10","wind","gust","10fg")):
                daily[v+"_max"] = data.resample(time="1D").max()
            else:
                daily[v+"_mean"] = data.resample(time="1D").mean()

        # winsorize 1–99% minden változóra
        for v in daily.data_vars:
            vals = daily[v].values
            daily[v].values = winsorize(vals)

        region_dailies.append(daily)
        ds.close()

    if len(region_dailies)==0:
        print(f"Nincs feldolgozható adat: {region}")
        continue

    # egyesítés idő szerint
    merged = xr.concat(region_dailies, dim="time")
    merged = merged.sortby("time")

    # térbeli átlag -> idősor
    data_dict = {}
    for v in merged.data_vars:
        spatial_dims = [d for d in merged[v].dims if d not in ["time"]]
        data_dict[v+"_mean"] = merged[v].mean(dim=spatial_dims).to_pandas()

    df = pd.DataFrame(data_dict)
    df.index.name = "date"

    # mentés
    nc_out = os.path.join(OUT_DIR, f"{region}_daily_light.nc")
    csv_out = os.path.join(OUT_DIR, f"{region}_daily_light.csv")
    merged.to_netcdf(nc_out)
    df.to_csv(csv_out)
    print(f"{region}: kész → {csv_out}")

print("\nPipeline lefutott. Ellenőrizd az era5_processed_light mappát.")


{'dania': 8, 'nemet': 8, 'tajvan': 12, 'uk': 8, 'usa': 13}

=== Feldolgozás: dania ===


dania fájlok: 100%|██████████| 8/8 [04:55<00:00, 36.88s/it]


dania: kész → /content/era5_processed_light/dania_daily_light.csv

=== Feldolgozás: nemet ===


nemet fájlok: 100%|██████████| 8/8 [05:12<00:00, 39.04s/it]


nemet: kész → /content/era5_processed_light/nemet_daily_light.csv

=== Feldolgozás: tajvan ===


...

tajvan: kész → /content/era5_processed_light/tajvan_daily_light.csv

=== Feldolgozás: uk ===


uk fájlok: 100%|██████████| 8/8 [05:05<00:00, 38.17s/it]


uk: kész → /content/era5_processed_light/uk_daily_light.csv

=== Feldolgozás: usa ===


usa fájlok: 100%|██████████| 13/13 [05:25<00:00, 25.00s/it]
/tmp/ipython-input-4246780395.py:89: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'latitude' ('latitude',) The recommendation is to set join explicitly for this case.
  merged = xr.concat(region_dailies, dim="time")
/tmp/ipython-input-4246780395.py:89: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'longitude' ('longitude',) The recommendation is to set join explicitly for this case.
  merged = xr.concat(region_dailies, dim="time")


usa: kész → /content/era5_processed_light/usa_daily_light.csv

Pipeline lefutott. Ellenőrizd az era5_processed_light mappát.
